In [1]:
import cv2
import tensorflow as tf
from keras.models import load_model
import pickle
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.preprocessing import image
import numpy as np
import random
import keyboard
from time import sleep
from textblob import TextBlob
from spellchecker import SpellChecker


In [2]:
dic1={0:"A",1:"B",2:"C",3:"D",4:"E",5:"F",6:"G",7:"H",8:"I",10:"K",11:"L",12:"M",13:"N",14:"O",15:"P",16:"Q",17:"R",18:"S",19:"T",20:"U",21:"V",22:"W",23:"X",24:"Y"}
dic2={0:"A",1:"E",2:"M",3:"S"}

In [3]:
model1 = load_model('sign_language_prediction_model.h5')
model2=load_model('second_model.h5')
def load_image1(filename,new_model1):
    img = image.load_img(filename,target_size = (28,28),color_mode = "grayscale")
    img = np.array(img)/255
    img = np.expand_dims(img,axis = 0) 
    img = img.reshape(1,28,28,1)
    result = new_model1.predict(img)
#     print("len",len(result[0]))
    result=np.argmax(result)
#     img_index = result.index(max(result))
#     print(result)
    if result>=9:
        result=result+1
    return dic1[result],result

def load_image2(filename,new_model2):
    img = image.load_img(filename,target_size = (28,28),color_mode = "grayscale")
    img = np.array(img)/255
    img = np.expand_dims(img,axis = 0) 
    img = img.reshape(1,28,28,1)
    result = new_model2.predict(img)
#     print("len",len(result[0]))
    result=np.argmax(result)
#     img_index = result.index(max(result))
#     print(result)
#     if result>=9:
#         result=result+1
    return dic2[result],result

In [ ]:
vid = cv2.VideoCapture(0)
spell = SpellChecker()
font = cv2.FONT_HERSHEY_SIMPLEX 
org = (70, 40) 
fontScale = 1.0
color = (255, 255, 0) 
thickness = 3
word=""
# pred1=" "
while True:
    ret,frame=vid.read()
    cv2.rectangle(frame, (20,80), (190,250), (0,0,255),2)
    
    img1=frame[80:250,20:190]
    cv2.imwrite("a.jpg", img1)
    pred1,prediction=load_image1('a.jpg',model1)
    if prediction ==0 or prediction==4 or prediction ==12 or prediction ==18 :
        pred1,prediction=load_image2('a.jpg',model2)
    if keyboard.is_pressed('f'):
        word=word+pred1
        sleep(0.1)
    if keyboard.is_pressed("s"):
        lst=word.split(' ')
        correct = spell.correction(lst.pop())
        lst.append(correct)
        word=" ".join(lst)
#         word=TextBlob(word)
#         word=word.correct()
#         print(word)
        word=word+" "
        sleep(0.1)
#         pred1=" "
    try:
        cv2.putText(frame, "Press F to add character to word and S to insert space between words", (50,30), font,  
                           0.5, (255,0,0), 1, cv2.LINE_AA)
        cv2.putText(frame, pred1, (85,60), font,  
                               fontScale, (0,0,255), thickness, cv2.LINE_AA)
        cv2.putText(frame,"App prediction is :-",(50,350),font,1,(0,255,0),3,cv2.LINE_AA)
        cv2.putText(frame, word, (50,400), font,  
                           1, (255,0,0), 3, cv2.LINE_AA) 
        cv2.imshow("frame",frame)
    except Exception as e:
#         print(e)
        cv2.imshow("frame",frame)
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
  
vid.release() 
cv2.destroyAllWindows()